In [1]:
from swebench.harness.utils import load_swebench_dataset
import json
import os
from tqdm.auto import tqdm
import json
import pandas as pd
import glob
instances = load_swebench_dataset(split="test", name="princeton-nlp/SWE-bench_Verified")

/Users/shrey/.local/share/virtualenvs/python-PN9waRBr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-26 22:02:31,691 - datasets - INFO - PyTorch version 2.4.0 available.


In [2]:
def get_trace(dfs, start_id, end_id):    
    trace = []
    for i in range(end_id, start_id - 1, -1):
        df = dfs[i]
        if len(df) == 1:
            trace.append(df["inputs"][0]["messages"] + [df["outputs"][0]])
        else:
            trace.append(df.iloc[-1]["inputs"]["messages"] + [df.iloc[-1]["outputs"]])
    return trace


In [3]:
def find_start_end(dfs, lens):
    start = -1
    for i, df in enumerate(dfs):
        if "You are a software engineer expert at solving bugs" in df.loc[0, "inputs"]["messages"][0]['kwargs']['content']:
            start = i
            break
    lens_copy = lens[start:]
    begin = -1
    end = -1
    for i, value in enumerate(lens_copy):
        if value != 1:
            if begin == -1:
                begin = i
            end = i
        elif begin != -1:
            break
    end = min(end, begin+2)
    end+=start
    return start, end

In [4]:
def get_final_patches():
    final_patches = {}
    for patch_path in glob.glob("/Users/shrey/Desktop/Composio-dev/composio/python/swe/agentic/langgraph_agent/logs/run_evaluation/langgraph_agent_final/composio/**/patch.diff"):
        project_name = patch_path.split("/")[-2]
        final_patches[project_name] = open(patch_path, "r").read()
    return final_patches

final_patches = get_final_patches()

In [5]:
def get_traces_from_folder(folder_path):
    run_dict = {}

    for x in tqdm(instances):
        project_name = x["instance_id"]
        if os.path.exists(f"./logs/{folder_path}/{project_name}/runs.json"):
            with open(f"./logs/{folder_path}/{project_name}/runs.json", "r") as f:
                runs_dict = json.load(f)
                df = pd.DataFrame(runs_dict)
                run_dict[project_name] = df

    for k,df in run_dict.items():
        dfs = []
        for _, _df in df.groupby(by="trace_id"):
            _df = _df.sort_values(by="start_time", ascending=True) 
            dfs.append(_df.reset_index(drop=True))
        dfs.sort(key=lambda x: x.iloc[0]['start_time'], reverse=True)
        run_dict[k] = dfs

    for k, dfs in run_dict.items():
        filtered_dfs = []
        for _df in dfs:
            if _df.iloc[0]["name"] == "BedrockChat": filtered_dfs.append(_df)
        if len(filtered_dfs) == 0:
            # print(k)
            pass
        else:
            run_dict[k] = filtered_dfs
    
    trace = {}
    probs = []
    for k, dfs in run_dict.items():
        lens = [len(_df) for _df in dfs]
        if k == "django__django-16901":
            start, end = 7, 13
        elif k == "sympy__sympy-11618":
            start, end = 7, 13
        else:
            start, end = find_start_end(dfs, lens)
        if start == -1:
            probs.append(k)
            continue
        if not (final_patches[k] in dfs[start].loc[0, "inputs"]["messages"][1]["kwargs"]["content"]):
            probs.append(k)
            continue
        tr = get_trace(dfs, start, end)
        trace[k] = tr

    return run_dict, trace, probs

In [6]:
run_dict, trace, probs = get_traces_from_folder("runs")

100%|██████████| 500/500 [01:38<00:00,  5.10it/s]



<!-- dfs[start].loc[0, "inputs"]["messages"][0]['kwargs']['content'] -->


In [7]:
probs

['django__django-10554',
 'django__django-10914',
 'django__django-11211',
 'django__django-11951',
 'django__django-12325',
 'django__django-12713',
 'django__django-13089',
 'django__django-13658',
 'django__django-13810',
 'django__django-13820',
 'django__django-13821',
 'django__django-15569',
 'django__django-16454',
 'django__django-16612',
 'django__django-16819',
 'django__django-16938',
 'django__django-17084',
 'psf__requests-2931',
 'pydata__xarray-3095',
 'scikit-learn__scikit-learn-25232',
 'sphinx-doc__sphinx-9229']

In [8]:
run_dict2, trace2, probs2 = get_traces_from_folder("runs2")

100%|██████████| 500/500 [00:00<00:00, 727.80it/s]


In [10]:
trace2.keys()

dict_keys(['django__django-10554', 'django__django-10914', 'django__django-11211', 'django__django-11951', 'django__django-12325', 'django__django-12713', 'django__django-13089', 'django__django-13658', 'django__django-13810', 'django__django-13820', 'django__django-13821', 'django__django-15569', 'django__django-16454', 'django__django-16612', 'django__django-16819', 'django__django-16938', 'django__django-17084', 'psf__requests-2931', 'pydata__xarray-3095', 'sphinx-doc__sphinx-9229'])

In [11]:
final_patch = get_final_patches()
trace.update(trace2)

In [12]:
final_patch = get_final_patches()
print(len(final_patch))
print(len(trace))

500
499


In [13]:
for k,tr in trace.items():
    last_message = tr[-1][1]['kwargs']['content']
    if final_patch[k] not in last_message:
        print(k)


In [14]:
os.makedirs("./logs/trajs", exist_ok=True)
for k,tr in trace.items():
    with open(f"./logs/trajs/{k}_traj.json", "w") as f:
        json.dump(tr, f, indent=2)


In [15]:
final_patch = get_final_patches()

for file_path in glob.glob("./logs/trajs/*.json"):
    # print(file_path)
    with open(file_path, "r") as f:
        tr = json.load(f)
    proj = file_path.split("/")[-1][:-10]
    if final_patch[proj] not in tr[-1][1]['kwargs']['content']:
        print(proj)

    


scikit-learn__scikit-learn-25232


In [3]:


run_dict = {}

for x in tqdm(instances):
    project_name = x["instance_id"]
    with open(f"./logs/runs/{project_name}/runs.json", "r") as f:
        runs_dict = json.load(f)
    df = pd.DataFrame(runs_dict)
    run_dict[project_name] = df


100%|██████████| 500/500 [01:19<00:00,  6.26it/s]


In [4]:
for k,df in run_dict.items():

    dfs = []
    for _, _df in df.groupby(by="trace_id"):
        ## sort the values based on start_time
        _df = _df.sort_values(by="start_time", ascending=True) 
        dfs.append(_df.reset_index(drop=True))

    # Sort dfs based on start_time of first row of each df
    dfs.sort(key=lambda x: x.iloc[0]['start_time'], reverse=True)
    run_dict[k] = dfs


In [5]:
for k, dfs in run_dict.items():
    filtered_dfs = []
    for _df in dfs:
        if _df.iloc[0]["name"] == "BedrockChat": filtered_dfs.append(_df)
    if len(filtered_dfs) == 0:
        print(k)
    else:
        run_dict[k] = filtered_dfs


django__django-11951
django__django-13810


In [6]:
def find_start_end(dfs, lens):
    start = -1
    end = -1
    for i, value in enumerate(lens):
        if value != 1:
            if start == -1:
                start = i
            end = i
        elif start != -1:
            break
    end = min(end, start+2)
    p = start - 1
    final = 0
    while p >= 0:
        if 'test' not in dfs[p].loc[0, "inputs"]["messages"][0]['kwargs']['content']:
            final = p
            break
        if lens[p] != 1:
            final = p+1
            break
        p -= 1
    
    return final, end


In [7]:
def find_start_end(dfs, lens):
    ## trim the lens list to remove all the elements in the beginning till first time 1 is encounteered
    if 1 in lens:
        first_one = lens.index(1)
        lens_copy = lens[first_one:]
    else: 
        first_one = 0
        lens_copy = lens
    start = -1
    end = -1
    for i, value in enumerate(lens_copy):
        if value != 1:
            if start == -1:
                start = i
            end = i
        elif start != -1:
            break
    end = min(end, start+2)
    start+=first_one
    end+=first_one
    p = start - 1
    final = 0
    while p >= 0:
        if 'test' not in dfs[p].loc[0, "inputs"]["messages"][0]['kwargs']['content']:
            final = p
            break
        if lens[p] != 1:
            final = p+1
            break
        p -= 1
    
    return final, end


In [15]:
import array
import glob
final_patches = {}
for patch_path in glob.glob("/Users/shrey/Desktop/Composio-dev/composio/python/swe/agentic/langgraph_agent/logs/run_evaluation/langgraph_agent_final/composio/**/patch.diff"):
    project_name = patch_path.split("/")[-2]
    final_patches[project_name] = open(patch_path, "r").read()


In [22]:
def find_start_end(dfs, lens):
    ## trim the lens list to remove all the elements in the beginning till first time 1 is encounteered

    start = -1
    for i, df in enumerate(dfs):
        if "You are a software engineer expert at solving bugs" in df.loc[0, "inputs"]["messages"][0]['kwargs']['content']:
            start = i
            break

    lens_copy = lens[start:]
    begin = -1
    end = -1
    for i, value in enumerate(lens_copy):
        if value != 1:
            if begin == -1:
                begin = i
            end = i
        elif begin != -1:
            break
    
    end = min(end, begin+2)
    end+=start
    return start, end


In [17]:
trace = {}
probs = []
for k, dfs in run_dict.items():
    lens = [len(_df) for _df in dfs]
    start, end = find_start_end(dfs, lens)
    if start == -1:
        probs.append(k)
        continue
    if not (final_patches[k] in dfs[start].loc[0, "inputs"]["messages"][1]["kwargs"]["content"]):
        probs.append(k)
        continue
    tr = get_trace(dfs, start, end)
    trace[k] = tr

In [19]:
probs

['django__django-11951',
 'django__django-12325',
 'django__django-12713',
 'django__django-13089',
 'django__django-13658',
 'django__django-13810',
 'django__django-13820',
 'django__django-13821',
 'django__django-15569',
 'django__django-16454',
 'django__django-16612',
 'django__django-16819',
 'django__django-16901',
 'django__django-16938',
 'django__django-17084',
 'psf__requests-2931',
 'pydata__xarray-3095',
 'scikit-learn__scikit-learn-25232',
 'sphinx-doc__sphinx-10466',
 'sphinx-doc__sphinx-9229',
 'sympy__sympy-11618']

In [18]:
for prob in probs:
    print("\"" + prob + "\"")

"django__django-11951"
"django__django-12325"
"django__django-12713"
"django__django-13089"
"django__django-13658"
"django__django-13810"
"django__django-13820"
"django__django-13821"
"django__django-15569"
"django__django-16454"
"django__django-16612"
"django__django-16819"
"django__django-16901"
"django__django-16938"
"django__django-17084"
"psf__requests-2931"
"pydata__xarray-3095"
"scikit-learn__scikit-learn-25232"
"sphinx-doc__sphinx-10466"
"sphinx-doc__sphinx-9229"
"sympy__sympy-11618"


In [19]:
dfs = run_dict["astropy__astropy-14539"]
lens = [len(_df) for _df in dfs]
start, end = find_start_end(dfs, lens)
dfs[start].loc[0, "inputs"]["messages"][0]['kwargs']['content']



'You are expert at reading test responses.'